Input Folder: root_20191213
Output Folder: final_cells

Import cell data and filters datasets on total fluorescence/expression level

Applies peakfinding on the 'g500' (fluorescence image with gain500) datadata with threshold 7500 and
adds it to the cells as 'storm' dataset with name 'storm_thd_7500'

In [1]:
import operator
import os
from collections import namedtuple
from functools import reduce

import numpy as np
import seaborn as sns
from addict import Dict
from colicoords import save
from dask.distributed import Client
from scipy.signal import medfilt
from skimage.feature import peak_local_max

from base_funcs import load_dirs_to_dict
sns.set_style('ticks')

In [2]:
client = Client('127.0.0.1:8786')
client

<Client: 'tcp://127.0.0.1:8786' processes=6 threads=12, memory=31.88 GiB>

In [22]:
cell_dict = load_dirs_to_dict('../data/root_20191213', client, regex='filtered')

In [23]:
CellMeasurement = namedtuple('SpotsMeasurement', ['cells', 'path', 'arr'])

In [24]:
# Iterate through the data and determine fluorescence intensity and number of spots
def r_dict(d, d_out, path):
    pth = os.path.join(*path)
    if not os.path.exists(pth):
        os.mkdir(pth)
    for k, v in d.items():
        if isinstance(v, Dict):
            r_dict(v, d_out, path + [k])
        else:
            print(path, k)
            intensity = v.get_intensity()
            spots = np.array([len(c.data.data_dict['storm_thd_7500']) for c in v])

            arr = np.dstack([intensity, spots])

            sm = CellMeasurement(arr=arr, path=path[1:], cells=v)
            parent = reduce(operator.getitem, path[1:], d_out)
            parent[k] = sm

big_dict = Dict()
r_dict(cell_dict, big_dict, ['root'])

['root', 'aht20', '20190930'] c41_filtered
['root', 'aht20', '20190930'] epec_filtered
['root', 'aht20', '20191001'] c41_filtered
['root', 'aht20', '20191001'] epec_filtered
['root', 'aht20', '20191002_03'] c41_filtered
['root', 'aht20', '20191002_03'] epec_filtered
['root', 'aht20', '20191002_04'] c41_filtered
['root', 'aht20', '20191002_04'] epec_filtered
['root', 'aht20', '20191002_04'] escc_filtered
['root', 'aht20', '20191023'] c41_filtered
['root', 'aht25', '20191010_01'] c41_filtered
['root', 'aht25', '20191010_02'] c41_filtered
['root', 'aht25', '20191010_03'] c41_filtered
['root', 'aht25', '20191023'] c41_filtered
['root', 'aht30', '20191005_02'] c41_filtered
['root', 'aht30', '20191005_03'] c41_filtered
['root', 'aht30', '20191009_01'] c41_filtered
['root', 'aht30', '20191009_02'] c41_filtered
['root', 'aht30', '20191022'] epec_filtered
['root', 'aht40', '20191004_02'] c41_filtered
['root', 'aht40', '20191004_03'] c41_filtered


In [25]:
def yield_class(d):
    for k, v in d.items():
        if isinstance(v, Dict):
            yield from yield_class(v)
        else:
            yield (k, v)

In [26]:
def check_int(v):
    i, s = v.arr.squeeze().T
    if (i.mean() > 750) and (i.mean() < 2400):
        return True
    else:
        return False

In [27]:
SC = Dict()
for k, v in yield_class(big_dict):
    if k.startswith('escc'):
        continue
    data = v.arr.squeeze()
    path = v.path
    cells = v.cells
    identifier = '_'.join(path) + '_' + k
    d = SC[identifier]
    d['arr'] = data
    d['path'] = path
    d['cells'] = cells
    d['name'] = k


In [31]:
dd = {k: v for k, v in SC.items() if check_int(v)}
for k in dd.keys():
    print(k)

aht20_20190930_epec_filtered
aht20_20191001_epec_filtered
aht20_20191002_03_epec_filtered
aht20_20191002_04_epec_filtered
aht20_20191023_c41_filtered
aht25_20191010_01_c41_filtered
aht25_20191010_02_c41_filtered
aht25_20191010_03_c41_filtered
aht25_20191023_c41_filtered
aht30_20191005_02_c41_filtered
aht30_20191009_01_c41_filtered
aht30_20191009_02_c41_filtered
aht30_20191022_epec_filtered


# Peakfinding

In [ ]:
storm_dtype = [('x', float), ('y', float), ('intensity', float), ('frame', int)]
def add_peakfind(cell, med=5, thd=7500, min_dst=5):
    """Add a storm dataset to the cell with locations of fluorescence peaks"""
    img = cell.data.data_dict['g500']
    mf = medfilt(img, med)
    img_bg = img - mf
    cell.data.add_data(img_bg, 'fluorescence', 'flu_mf_{}'.format(med))

    peaks = peak_local_max(img_bg, min_distance=min_dst, threshold_abs=thd)
    y, x = peaks.T

    new_storm = np.empty(len(x), dtype=storm_dtype)
    new_storm['x'] = x
    new_storm['y'] = y
    new_storm['intensity'] = np.ones_like(x)
    new_storm['frame'] = np.ones_like(x)

    cell.data.add_data(new_storm, 'storm', 'new_thd_{}'.format(thd))

In [ ]:
peak_cells = {}
for k, v in dd.items():
    print(k)
    cells = v.cells.copy()
    _ = [add_peakfind(c) for c in cells]
    peak_cells[k] = cells

In [ ]:
for k, v in peak_cells.items():
    save('../data/final_cells/' +  k + '.hdf5', v)